This tutorial is part-2 of [Comprehensive tutorial on NLP](https://www.kaggle.com/kksienc/comprehensive-nlp-tutorial-1-ml-perspective) series. In this part we will learn about word embeddings and see how Deep learning has simplified NLP. 

Pre-requisite: Basic Deep learning understanding would be helpful though not mandatory.

<a class="kk" id="0.1"></a>
## Contents

1. [Introduction to Word Embedding](#1)
    1. [Dimensionality](#1.1) 
    1. [Padding](#1.2)
    1. [Euclidean Distance](#1.3)
    1. [Cosine Similarity](#1.4)
1. [Word Embedding Techniques](#2)
    1. [Word2Vec](#2.1)
        1. [Skip-Gram](#2.1.1)
        1. [CBOW (Continuous Bag of Words)](#2.1.2)
    1. [GloVe](#2.2) 
    1. [FastText](#2.3)
1. [Text to Numeric Convertion Using Word Vectors](#3)
    1. [Vector Averaging](#3.1)
        1. [Vector Averaging With Word2Vec](#3.1.1)
        1. [Vector Averaging With GloVe](#3.1.2)
        1. [Vector Averaging With FastText](#3.1.3) 
    1. [Embedding Matrix & Keras Embedding layer](#3.2)
        1. [Word2Vec Embedding layers](#3.2.1)
        1. [GloVe Embedding layers](#3.2.2)
        1. [FastText Embedding layers](#3.2.3)      
1. [Deep Learning models](#4)
    1. [Basic-DNN](#4.1)
    1. [CNN](#4.2)
    1. [RNN](#4.3)
    1. [Recurrent Neural Network -LSTM](#4.4)
    1. [Recurrent Neural Network – GRU](#4.5)
 

# 1. Introduction to Word Embedding  <a class="kk" id="1"></a>
[Back to Contents](#0.1)

Word Embedding is also known as Word Vectorization. It means converting word into vector. Vectors are numeric representation of a point in space. Mathematically vectors are 1D array or sequence of numbers.  


<B>Why we need Word Embedding? </B>

A problem with our previous text to numeric conversion techniques (countvectorizer & tfidf) was that, they ignore synonyms. For example, word 'measure' and ‘calculate’ were represented differently however, in most sentences they can be used interchangeably. In Word Embedding, similar words are spatially close to each other in vector space. Word Embedding is also capable of preserving semantic and syntactic similarity and relation with other words. The vector representation are such that geometric transformation adopts syntax and semantic. For instance, by adding a “female” vector to the vector "king", we obtain the vector "queen" and by adding a "plural" vector to the vector "king", we obtain "kings". 

Another problem we observe in [part-1](https://www.kaggle.com/kksienc/comprehensive-nlp-tutorial-1-ml-perspective), was production of high dimensionality sparse matrix. Word Embedding solve this also, as it produces low dimensional dense matrix.


In [ ]:
from IPython.display import Image
Image("/kaggle/input/images/textproc.jpg",  width=400)

Before applying Word Embedding techniques lets look into into few common NLP vocabulary terms.

## 1.1 Dimensionality  <a class="kk" id="1.1"></a>

Dimensionality refers to the length of vectors and equals to number of columns of vector representation.
For example, in the above figure each token is encoded into vector of dimesionality 3.

## 1.2 Padding <a class="kk" id="1.2"></a>

Padding is task of appending string up to given specific length with whitespaces. Padding is used to represent all records as fixed length.

## 1.3 Euclidean Distance <a class="kk" id="1.3"></a>

Euclidean distance is the shortest distance between two points in (Euclidean) space.

 

In [ ]:
Image("/kaggle/input/images/eucldeandistance.png",  width=400)

## 1.4 Cosine Similarity  <a class="kk" id="1.4"></a>
Cosine similarity is a measure of similarity between two nonzero vectors of an inner product space. It measures the cosine of the angle between them. 
 

In [ ]:
Image("/kaggle/input/images/cosineSimilarity.png",  width=400)

# 2. Word Embedding Techniques <a class="kk" id="2"></a>
[Back to Contents](#0.1)

Now we will dive deep into word Embedding techniques. But first let's fetch our [Disaster Tweets](https://www.kaggle.com/c/nlp-getting-started) dataset and clean it, as we did in [part 1](https://www.kaggle.com/kksienc/comprehensive-nlp-tutorial-1-ml-perspective).

In [ ]:
# Fetch & clean dataset 
!pip install pyspellchecker
from spellchecker import SpellChecker
import pandas as pd
from nltk.corpus import stopwords 
from nltk.corpus import wordnet

from nltk.stem import WordNetLemmatizer 
import nltk 
import re
import numpy as np  
import pandas as pd 
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")


def convert_to_antonym(sentence):
    words = nltk.word_tokenize(sentence)
    new_words = []
    temp_word = ''
    for word in words:
        antonyms = []
        if word == 'not':
            temp_word = 'not_'
        elif temp_word == 'not_':
            for syn in wordnet.synsets(word):
                for s in syn.lemmas():
                    for a in s.antonyms():
                        antonyms.append(a.name())
            if len(antonyms) >= 1:
                word = antonyms[0]
            else:
                word = temp_word + word # when antonym is not found, it will
                                    # remain not_happy
            
            temp_word = ''
        if word != 'not':
            new_words.append(word)
    return ' '.join(new_words)


# def correct_spellings(text):
#     spell = SpellChecker()
#     corrected_words = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_words.append(spell.correction(word))
#         else:
#             corrected_words.append(word)
#     return " ".join(corrected_words)

        
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV }
def lemma_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word ,wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

 
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
  """
    text = text.lower() # lowercase text
    text= re.sub(r'[^\w\s#]',' ',text) #Removing every thing other than space, word and hash
    text  = re.sub(r"https?://\S+|www\.\S+", "", text )
    text= re.sub(r'[0-9]',' ',text)
    #text = correct_spellings(text)
    text = lemma_words(text)
    text = convert_to_antonym(text)
    text = re.sub(' +', ' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text    
    return text


train_df['text'] = train_df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)

sentences= pd.DataFrame(columns=['text'])
sentences['text']= pd.concat([train_df["text"], test_df["text"]])

from collections import defaultdict
tokens_list = [row.split() for row in sentences['text']]



## 2.1 Word2Vec    <a class="kk" id="2.1"></a>

 Word2Vec is group of related models that are used to produce Word Embeddings. It was created & patented by Tomas Mikolov and a group of a research team from Google in 2013. Each unique word in the corpus is assigned a corresponding vector in the space. Word2Vec relies only on local information of language hence the semantics learnt for a given word is only affected by the surrounding words. Underlying assumption of Word2Vec is that two words sharing similar contexts also share a similar meaning, this at times results into similar vector representation (cosine similarity) of multiple words. One more drawback of word2vec is it's unablity to take care of OutofVocabulary(OOV) word.


In [ ]:
Image("/kaggle/input/images/word2vec.png",  width=450)


Word2Vec comes in two flavours,
 - Skip-Gram and 
 - Continuous Bag of Words (CBOW)

Underneath, Word2Vec uses neural network algorithms that can be trained on any type of sequential data. Fortunately we have libraries available that have already implemented these algorithms and we have to just call the method with proper arguments. A popular one among such libraries is <B>gensim</B>. It provides the [Word2Vec Class](https://radimrehurek.com/gensim/models/word2vec.htm) for working with a Word2Vec model.

For more details of Gensim implementation of Wrod2Vec refer their [official documentation](https://radimrehurek.com/gensim/models/word2vec.html)


###  2.1.1 Skip-Gram  <a class="kk" id="2.1.1"></a>
 
Skip-Gram is designed to predict the context from base word. From a given word, Skip-gram model tries to predict its neighbouring words.   

Skip-gram is a [(target, context), relevancy] generator. Skip-gram generator gives us pair of words and their relevance (a float value). Let's generate Word2Vec skip-gram embedding for our cleaned-up text corpus using gensim.

In [ ]:
Image("/kaggle/input/images/skipgram.png",  width=450)

#### Building Skipgram  WordVectors using gensim

In [ ]:
from gensim.models import Word2Vec
from time import time
t = time()
# initialize skipgram model
sg_model = Word2Vec(min_count=2,
                    window=2,size=300, sg = 1,
                    sample=5e-5, alpha=0.05, 
                    min_alpha=0.0005,negative=20 )

# build model vocabulary
sg_model.build_vocab(tokens_list)

# train the model
sg_model.train(tokens_list, total_examples=sg_model.corpus_count, epochs=30, report_delay=1)

print('Time to build Skip gram model vocab: {} mins'.format(round((time() - t) / 60, 2)))

We have just build our first word-embedding model and that also with only 3 lines of code. Lets play with this model.

##### Vector representation of word

In [ ]:
sg_model.wv.__getitem__('hope')

##### Validate dimension of our word vector

In [ ]:
len(sg_model.wv.__getitem__('hope'))

##### Measure similarity   b/w two words 

In [ ]:
sg_model.wv.similarity('people','saint' )


In [ ]:
sg_model.wv.similarity('people', 'terrorist')

Intrestingly, there's higher similarity b/w people & terrorists than people & saints :()


##### Fetch most similar words for any given word 

In [ ]:
# top 5 similar words
sg_model.wv.most_similar('fire')[:5]

##### Fetch list of word vocabulary 

In [ ]:
print(list(sg_model.wv.vocab))
 

### 2.1.2 CBOW (Continuous Bag of Words)  <a class="kk" id="2.1.2"></a>

CBOW is designed to predict the base(target) word from context. CBOW is faster to train than the skip-gram and gives slightly better accuracy for frequent words.


In [ ]:
Image("/kaggle/input/images/cbow.png", width=450)

In [ ]:
#### Building CBOW wordvectors
from gensim.models import Word2Vec
from time import time
t = time()
# initialize
cbow_model = Word2Vec(min_count=2,window=2,size=300, sg = 0,sample=5e-5, alpha=0.05, min_alpha=0.0005, 
                     negative=20 )
# build model vocabulary
cbow_model.build_vocab(tokens_list)

# train the model
cbow_model.train(tokens_list, total_examples=cbow_model.corpus_count, epochs=30, report_delay=1)

print('Time to build CBOW model vocab: {} mins'.format(round((time() - t) / 60, 2)))


Did you notice CBOW trained faster than Skipgram ?


####  Pretrain Word2Vec

Google has made available pretrained word embedding which includes word vectors for a vocabulary of 3 million words and phrases that they have trained on roughly 100 billion words from Google News dataset using Word2Vec.

In [ ]:
#fetching  pretrain wordvector
from gensim.models.keyedvectors import KeyedVectors
t = time()
pretrained_w2vec_embedding = KeyedVectors.load_word2vec_format('../input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin', binary=True)
print('Time to fetch  pretrain  Word2Vec model vocab: {} mins'.format(round((time() - t) / 60, 2)))

#### vector representation of Word2Vec pretrained word 

In [ ]:
pretrained_w2vec_embedding['people']

In [ ]:
# shape of pretrain w2vec embedding
pretrained_w2vec_embedding.vectors.shape

In [ ]:
#del pretrained_w2vec_embedding

## 2.2 GloVe  <a class="kk" id="2.2"></a>

[GloVe](https://nlp.stanford.edu/pubs/glove.pdf) stands for "Global Vectors". It is a Word Embedding [project](https://nlp.stanford.edu/projects/glove/)  written in C language and developed by Stanford university researchers in 2014. Glove embedding technique is based on 
firstly, construction of a co-occurrence matrix from a training corpus and then 
secondly, factorization of co-occurrence matrix in order to yield word vector.

Unlike word2vec, which captures only local statistics of token, Glove captures both global statistics and local statistics of a text tokens. GloVe embeddings relate to the probabilities that two words appear together. [glove_python](https://github.com/maciejkula/glove-python) library provides glove implementation.

For more details of GloVe implementation of  glove_python refer their [official documentation](https://pypi.org/project/glove/)


In [ ]:
!pip install glove_python

In [ ]:
#importing the glove library
from glove import Corpus, Glove

# creating a corpus object
corpus = Corpus() 

#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(tokens_list, window=3)

#creating a Glove object which will use the matrix created in the above lines to create embeddings
#we can set the learning rate as glove uses Gradient Descent
glove = Glove(no_components=300, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

#####  Displaying Glove WordVector of a word 

In [ ]:
glove.word_vectors[glove.dictionary['people']]

#### Pretrain Glove

Glove developers have also made available pre-computed embeddings for millions of English tokens, obtained from training Wikipedia data and Common crawl data.

<B>Due to notebook memory constrains, I have commented out Pretrained Glove and Pretrained Fast-text.However, there usage is similar to pre-trained Word2Vec. You can uncomment and run on separate notebook </B>

In [ ]:
# # Fetch pretrain glove word vectors 
# import numpy as np 
# pretrained_glove_embedding={}
# with open('../input/nlpword2vecembeddingspretrained/glove.6B.300d.txt','r') as f:
#     for line in f:
#         values=line.split()
#         word=values[0]
#         vectors=np.asarray(values[1:],'float32')
#         pretrained_glove_embedding[word]=vectors
# f.close()

#####   Glove pretrained WordVector of a word 

In [ ]:
#pretrained_glove_embedding['hello']


In [ ]:
# no of  word in pretrained glove embedding
#len(pretrained_glove_embedding)

In [ ]:
#del pretrained_glove_embedding

## 2.3. Fast-Text <a class="kk" id="2.3"></a>

[FastText](https://fasttext.cc/) is a library for learning of word embeddings and text classification. The Facebook Research Team created fastText in Nov 2015. Fast-Text is an extension of word2vec library. It builds on Word2Vec by learning vector representations for each word and the n-grams found within each word. FastText assumes a word to be formed by a n-grams of character. For example, sunny is composed of [sun, sunn,sunny],[sunny,unny,nny]... etc, where n could range from 1 to the length of the word. The values of the representations are then averaged into one vector at each training step. While this adds a lot of additional computation to training it enables word embeddings to encode sub-word information. Thus even for previously unseen words, typo errors, and OOV (Out Of Vocabulary) words the model can make an educated guess towards its meaning.Obvious trade off is processing time. Gensim provides the [FastText implementation](https://radimrehurek.com/gensim/models/fasttext.html).

For more details of FastText implementation of  Gensim refer their [official documentation](https://radimrehurek.com/gensim/models/fasttext.html)



In [ ]:
# dimension =300
# from gensim.models import FastText
# fasttext_model = FastText(tokens_list, size=dimension, window=5, min_count=5, workers=4, sg=1)


#####  Displaying FastText WordVector of given word 

In [ ]:
# fasttext_model.wv.__getitem__('people')

In [ ]:
# # fasttext word similarity measure
# fasttext_model.wv.similarity('evacuation','shelter' )

In [ ]:
# # most similar words
# fasttext_model.wv.most_similar('fire')[:5]

##### PreTrained Fasttext

FastText developers have also made available pre-computed embeddings for millions of english tokens, obtained from training Wikipedia data and common crawl data. 

Disclaimer: Loading the fastText pretrain will consume some serious memory.

As mention earlier, due to notebook memory constrains, I have commented out Pretrained-Glove and Pretrained-Fasttext here.The usage is quite similar to pre-trained Word2Vec. 

In [ ]:
# def get_coefs(word, *arr): 
#     return word, np.asarray(arr, dtype='float32')

# EMBEDDING_FILE = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'
# pretrained_fasttext_embedding = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in (open(EMBEDDING_FILE)))

In [ ]:
## no of words in pretrained fasttext embedding
#len(pretrained_fasttext_embedding)

#####  Displaying FastText pretrained WordVector of a word 

In [ ]:
#pretrained_fasttext_embedding['fire']

In [ ]:
#len(pretrained_fasttext_embedding['fire'])

# 3. Text to Numeric Convertion Using Word Vectors <a class="kk" id="3"></a>
[Back to Contents](#0.1)

We learned about word embedding techniques and created word vectors for our corpus.  Now we will convert our textual data into numerical using these word vectors. I will explain about two popular texts to numerical conversion techniques using word vectors, 

1. Vector Averaging  
1. Embedding Matrix and Keras Embedding layer


## 3.1 Vector Averaging  <a class="kk" id="3.1"></a>
In Vector Averaging approach we average all word embeddings (of words) in the text. Final length remains equal to word vector dimension. This is go to technique when we are planning to use machine learning models such a logistic regression, naïve-bayes, svm etc.  
 

### Vector Averaging With Word2Vec <a class="kk" id="3.1.1"></a>

In [ ]:
# functions for Vector Averaging with word2Vec
import numpy as np
def w2v_embeddings(text,w2v_model,dimension):
    if len(text) < 1:
        return np.zeros(dimension)
    else:
        vectorized = [w2v_model.wv[word] if word in w2v_model.wv else np.random.rand(dimension) for word in text] 
    
    sum = np.sum(vectorized,axis=0)
    ## return the average
    return sum/len(vectorized)     

def get_w2v_embeddings(text,w2v_model,dimension):
        embeddings = text.apply(lambda x: w2v_embeddings(x, w2v_model,dimension))
        return list(embeddings)

#### SkipGram Model

In [ ]:
# Text to Numeric Vector Averaging using sgmodel 
train_embeddings_sg_model  = get_w2v_embeddings(train_df['text'],sg_model,dimension=300)
 

In [ ]:
# Validate train set size
len(train_embeddings_sg_model)

In [ ]:
# validate dimension
len(train_embeddings_sg_model[0])

#### CBOW model

In [ ]:
# Text to numeric Vector Averaging using cbow model
train_embeddings_cbow_model_  = get_w2v_embeddings(train_df['text'],cbow_model,dimension=300)

### Vector Averaging With Glove <a class="kk" id="3.1.2"></a>

In [ ]:
## functions  for Vector Averaging with GloVe
import numpy as np
def glove_embeddings(text, glove_model, dim ):
    dic=glove_model.dictionary
    if len(text) < 1:
        return np.zeros(dim)
    else:
        vectorized = [glove_model.word_vectors[dic[word]] if word in dic else np.random.rand(dim) for word in text]  
    sum = np.sum(vectorized,axis=0)
    ## return the average
    return sum/len(vectorized)     

def get_glove_embeddings(text,glove_model,dimension):
        embeddings = text.apply(lambda x: glove_embeddings(x,glove_model, dimension))
        return list(embeddings)


In [ ]:
# Text to numeric Vector Averaging using glove
import numpy as np
train_embeddings_glove = get_glove_embeddings(train_df['text'],glove,dimension=300)
test_embeddings_glove = get_glove_embeddings(test_df['text'],glove,dimension=300)

### Vector Averaging With Fasttext  <a class="kk" id="3.1.3"></a>

As Fastext is an extension of word2vec hence the same averaging function of w2vec i.e.`get_ w2v_embeddings` will work for fasttext model too. 

In [ ]:
# ## Text to numeric using Averaging with Fasttext
# import numpy as np
# fasttext_train_embeddings = w2v_embeddings(train_df['text'], fasttext_model,dimension=300)
# fasttext_test_embeddings = w2v_embeddings(test_df['text'],  fasttext_model,dimension=300)

#### Vector Averaging With pretrained Embeddings

 

In [ ]:
def pretrained_embeddings(text,model,dimension):
    if len(text) < 1:
        return np.zeros(dimension)
    else:
        vectorized = [model[word] if word in model else np.random.rand(dimension) for word in text] 
    
    sum = np.sum(vectorized,axis=0)
    ## return the average
    return sum/len(vectorized)     

def get_pretrained_embeddings(text,model,dimension):
        embeddings = text.apply(lambda x: pretrained_embeddings(x, model,dimension))
        return list(embeddings)

In [ ]:
# uncomment section in below code in order to perfrom vector averaging using pretrained word embeeding

#Text to numeric Vector Averaging  using  pretrianed word2vec
train_embeddings_w2vec_pretrained_  = get_pretrained_embeddings(train_df['text'],pretrained_w2vec_embedding,dimension=300)

## Text to numeric Vector Averaging  using  pretrianed glove
# train_embeddings_glove_pretrained_  = get_pretrained_embeddings(train_df['text'],pretrained_glove_embedding,dimension=300)

## Text to numeric Vector Averaging  using  pretrianed fasttext
# train_embeddings_fasttext_pretrained_  = get_pretrained_embeddings(train_df['text'],pretrained_fasttext_embedding,dimension=300)



## 3.2 Embedded Matrix & Keras Embedding layer <a class="kk" id="3.2"></a>

Averaging is preferred choice when we intend to use Machine Learning models such as lr, svm, gbm etc. but our purpose here is to utilise Deep-learning algorithms. Deep Learning is a layer bases learning where each layer passes its learning to the next layer.   Few libraries have implement deep learning algorithms. A popular one among them is Keras. We will use Keras for our deep learning modelling purpose.

For text processing Keras offers an embedding layer. This is the first layer of deep learning algorithm. Weights of the Embedding layer are of the shape (vocabulary_size, embedding_dimension), this weight matrix is also called as Embedding matrix. We will first generate this embedding matrix from our word vectors and then initialize Keras embedding layer for each of our word embeddings. 

Moreover, Keras has built-in utilities for doing tokenization and encoding of text. We will use these utilities as they take care of a number of important features such as stripping special characters from strings, padding, fetching N most common words in dataset etc.

In [ ]:
# using keras built in utilities
# tokenizing using keras  tokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.layers import Embedding
tokenizer_obj=Tokenizer()
# to builds the word index
tokenizer_obj.fit_on_texts(tokens_list)
# to turns strings into lists of integer indices.
sequences=tokenizer_obj.texts_to_sequences(tokens_list)
# defining maximum length of sequence 
MAX_LEN= 50
# pad_sequences is used to ensure that all sequences in a list have the same length
tweet_pad= pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

# segregating text & train from corpu
x_train = tweet_pad[:7613]
x_test = tweet_pad[7613:]

targets =  [target for target in train_df['target']]

# set of all word and their sequence no
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))
vocab_size = len(word_index)+1


In [ ]:
# function to generate embeeding layer weights i.e. embeeding_matrix


def generate_word2vec_embeeding_matrix(word_vector_model, dimension, vocab_size= vocab_size, word_index =word_index):
    embedding_matrix=np.zeros((vocab_size,dimension))
    for word,i in tqdm(word_index.items()):
        if i > vocab_size:
            continue
        if word in word_vector_model.wv:  
            emb_vec=word_vector_model.wv.__getitem__(word)
            embedding_matrix[i]=emb_vec
    return embedding_matrix

def generate_pretrained_embeeding_matrix(word_vector_model, dimension, vocab_size= vocab_size, word_index =word_index):
    embedding_matrix=np.zeros((vocab_size,dimension))
    for word,i in tqdm(word_index.items()):
        if i > vocab_size:
            continue
        if word in word_vector_model:  
            emb_vec=word_vector_model[word]
            embedding_matrix[i]=emb_vec
    return embedding_matrix



Lets create Keras embedding layers for our word embeddings. As we have created 4 trained word embedding model (skipgram, cbow, glove and fasttext) and 3 pretrained model (one each for word2vec, glove and fasttext) for all these seven we will create a Keras embedding layer.

### Word2Vec Embedding layers  <a class="kk" id="3.2.1"></a>

##### Trained skipgram

In [ ]:
from keras.layers import Embedding

embedding_matrix_sg_trained = generate_word2vec_embeeding_matrix(sg_model, dimension = 300)

embedding_layer_sg_trained = Embedding(vocab_size, output_dim= 300, weights=[embedding_matrix_sg_trained], 
                                     input_length=MAX_LEN, trainable=False)

##### Pre-Trained  Word2Vec

In [ ]:
# pre trainde word2vec  
embedding_matrix_w2v_pretrained = generate_pretrained_embeeding_matrix(pretrained_w2vec_embedding, dimension =300)    

embedding_layer_w2v_pretrained = Embedding(vocab_size, output_dim= 300, weights=[embedding_matrix_w2v_pretrained], 
                                     input_length=MAX_LEN, trainable=False)

##### Trained  CBOW

In [ ]:
embedding_matrix_cbow_trained = generate_word2vec_embeeding_matrix(cbow_model, dimension = 300)

embedding_layer_cbow_trained = Embedding(vocab_size, output_dim= 300, weights=[embedding_matrix_cbow_trained], 
                                     input_length=MAX_LEN, trainable=False)

### GloVe Embedding Layers   <a class="kk" id="3.2.2"></a>

##### Trained Glove

In [ ]:
import numpy as np
embedding_matrix_glove_trained=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    if i > vocab_size:
        continue
    
    emb_vec=glove.word_vectors[glove.dictionary[word]]
    if emb_vec is not None:
        embedding_matrix_glove_trained[i]=emb_vec

In [ ]:
embedding_layer_glove_trained = Embedding(vocab_size, 300 , weights=[embedding_matrix_glove_trained], 
                                     input_length=MAX_LEN, trainable=False)

##### PreTrained Glove

In [ ]:
# embedding_matrix_glove_pretrained = generate_pretrained_embeeding_matrix(pretrained_glove_embedding, dimension =300)    

# embedding_layer_glove_pretrained = Embedding(vocab_size, output_dim= 300, weights=[embedding_matrix_glove_pretrained], 
#                                      input_length=MAX_LEN, trainable=False)

### Fasttext  Embedding layers  <a class="kk" id="3.2.3"></a>

##### Trained FastText

In [ ]:
 
# embedding_matrix_fasttext_trained = generate_word2vec_embeeding_matrix(fasttext_model, dimension =300)    

# embedding_layer_fasttext_trained = Embedding(vocab_size, output_dim= 300, weights=[embedding_matrix_fasttext_trained], 
#                                      input_length=MAX_LEN, trainable=False)

##### Pre-Trained FastText

In [ ]:
# embedding_matrix_fasttext_pretrained = generate_pretrained_embeeding_matrix(pretrained_fasttext_embedding, dimension =300)    

# embedding_layer_fasttext_pretrained = Embedding(vocab_size, output_dim= 300, weights=[embedding_matrix_fasttext_pretrained], 
#                                      input_length=MAX_LEN, trainable=False)

## 4. Deep Learning Models <a class="kk" id="4"></a>
[Back to Contents](#0.1)

 We have initialized Keras embedding layer for our various word embedding models. Now it’s time to train using deep learning models. I will demonstrate how to train for glove pertained layer. You can test with other six embedding layer also (by just reassigning embedding_layer). One point you will notice that pretrained embedding layers performs much better than their trained counter parts. Again the purpose here is to depict basic Deep Learning model performance and not to obtain high score. 

In [ ]:
# Declare embeeding layer of your choics 
embedding_layer = embedding_layer_sg_trained

# try with other embedding layes too
#embedding_layer_glove_pretrained
# embedding_layer_fasttext_pretrained
# embedding_layer_fasttext_trained
# embedding_layer_cbow_trained
# embedding_layer_sg_trained
# embedding_layer_w2vec_pretrained
# embedding_layer_glove_trained

In [ ]:
import keras
my_callbacks = [
    keras.callbacks.EarlyStopping(patience=5),
    
]

### 4.1 Basic DNN <a class="kk" id="4.1"></a>


In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
dnn_model = Sequential()
dnn_model.add(embedding_layer)
dnn_model.add(Flatten())
dnn_model.add(Dense(1, activation='sigmoid'))

dnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

dnn_model.summary()

history = dnn_model.fit(x_train,  y = targets,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2, callbacks= my_callbacks)

###  4.2 CNN <a class="kk" id="4.2"></a>

In [ ]:
#CNN is suitable for image processing
# import keras
# cnn_model = Sequential()
# # note : below we add embedding layer
# cnn_model.add(embedding_layer)
# cnn_model.add(keras.layers.Dropout(0.2))
# cnn_model.add(keras.layers.Conv1D(3,3, padding='valid',activation='relu', strides=1))
# cnn_model.add(keras.layers.GlobalMaxPooling1D())
# cnn_model.add(keras.layers.Dense(20))
# cnn_model.add(keras.layers.Dropout(0.2))
# cnn_model.add(keras.layers.Activation('relu'))
# cnn_model.add(keras.layers.Dense(1))
# cnn_model.add(keras.layers.Activation('sigmoid'))

# # Get model summary
# cnn_model.summary()
# cnn_model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['acc'])

# # compile the model
# history = cnn_model.fit(x_train,  y = targets,
#                     epochs=10,
#                     batch_size=32,
#                     validation_split=0.2)

### 4.3 RNN <a class="kk" id="4.3"></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
rnn_model = Sequential()
# note : below we add embedding layer
rnn_model.add(embedding_layer)
rnn_model.add(SimpleRNN(32))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = rnn_model.fit(x_train, y = targets,epochs=10, batch_size=32,validation_split=0.2)

### 4.4  Recurrent Neural Network -LSTM <a class="kk" id="4.4"></a>

In [ ]:
from keras.layers import LSTM
lstm_model = Sequential()
# note : below we add embedding layer
lstm_model.add(embedding_layer)
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = lstm_model.fit(x_train, y = targets,epochs=10, batch_size=32,validation_split=0.2)

### 4.5 Recurrent Neural Network – GRU <a class="kk" id="4.5"></a>

In [ ]:
from keras.layers import GRU
gru_model = Sequential()
# note : below we add embedding layer
gru_model.add(embedding_layer)
gru_model.add(GRU(32))
gru_model.add(Dense(1, activation='sigmoid'))

gru_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = gru_model.fit(x_train, y = targets,epochs=10, batch_size=32,validation_split=0.2)

###  Target Prediction  

In [ ]:
# lets predict target values for test set
model =  dnn_model
raw_preds = model.predict(x_test)
preds = raw_preds.round().astype(int)
preds

In [ ]:
Image("/kaggle/input/images/too much.png",  width=450)

 


Thanks for reading ! 

If you like this kernel, <b>  Do Upvote</b>!!! Its at the top right near Copy&Edit and will not cost you:)


In the next [part-3](https://www.kaggle.com/kksienc/comprehensive-nlp-tutorial-3-bert-others) we will read about state-of-the-art 'BERT Embedding'.





## References
- Deep Learning with Python by FRANÇOIS CHOLLET  http://faculty.neu.edu.cn/yury/AAI/Textbook/Deep%20Learning%20with%20Python.pdf
- https://en.wikipedia.org/
- https://www.oreilly.com/library/view/statistics-for-machine/9781788295758/eb9cd609-e44a-40a2-9c3a-f16fc4f5289a.xhtml
- https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-skip-gram.html
- https://www.kaggle.com/c/quora-question-pairs/discussion/31257#177483
- https://www.kaggle.com/slatawa/simple-implementation-of-word2vec
- https://becominghuman.ai/how-does-word2vecs-skip-gram-work-f92e0525def4
- https://www.thinkinfi.com/2019/06/single-word-cbow.html(image)
- https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer
- https://medium.com/@japneet121/word-vectorization-using-glove-76919685ee0b
- https://www.kaggle.com/christofhenkel/fasttext-starter-description-only
